In [3]:
do_posts = True
overwrite = True

In [4]:
import sys 
sys.path.append('..')
import logging as lg
import re
import numpy as np
import coloredlogs
import constants as CS
import pandas as pd
from helpers import *
from tqdm import tqdm
import helpers.globals_loader as globals_loader
import helpers.helper_functions as helper_functions
coloredlogs.install()

CS.USE_MINIFIED_DATA = False
CS.LOAD_POSTS = True
CS.LOAD_COMMENTS = False
CS.LOAD_FOUNDATIONS = False
CS.LOAD_LIWC = False
globals_loader.init()

2021-11-13 01:11:20 PHDesktop root[1632] INFO Loading posts: /home/philipp/Document/Coding/Reddit_Morality_Analysis/datasets/data/posts_cleaned_27_6_2021.csv
2021-11-13 01:11:31 PHDesktop root[1632] WARNING Skipping loading comments...
2021-11-13 01:11:31 PHDesktop root[1632] WARNING Skipping loading liwc...
2021-11-13 01:11:31 PHDesktop root[1632] WARNING Skipping loading foundations...
2021-11-13 01:11:31 PHDesktop root[1632] INFO Loading spacy


In [55]:
df = globals_loader.df_posts if do_posts else globals_loader.df_comments

In [6]:
prefix = "post" if do_posts else "comment"
df = globals_loader.df_posts if do_posts else globals_loader.df_comments
ids = df[prefix+"_id"].tolist()

reddit = globals_loader.reddit
elem_prefix = "t3" if do_posts else "t1"
ids = [i if i.startswith(elem_prefix+"_") else f"{elem_prefix}_{i}" for i in ids]
scores = []

post_idx = 0
comment_idx = 0


if do_posts:

    scores = np.zeros((len(ids),3))
    scores[:,0] = range(len(ids))
    scores = scores.tolist()
    
    scores_lst = []
    
    for submission in tqdm(reddit.info(ids), total=len(ids)):
        #submission = reddit.info(ids[i])
        updated = [ids[post_idx],0,0,""] #id, score, ratio
        updated[1] = submission.score
        updated[2] = submission.upvote_ratio
        updated[3] = submission.link_flair_text
        scores[post_idx] = updated
        
        scores_lst.append(updated)
        post_idx += 1
        
        # see https://api.reddit.com/api/info/?id=t3_apcnyn
else:
    
    scores = np.zeros((len(ids),2))
    scores[:,0] = range(len(ids))
    scores = scores.tolist()
    
    scores_lst = []

    for comment in tqdm(reddit.info(ids), total=len(ids)):
        #comment = reddit.info(ids[i])   

        updated = [comment_idx,0] #id, score, ratio
        updated[1] = comment.score
        scores[comment_idx] = updated
        scores_lst.append(updated)
        comment_idx+=1
        
        
        # comments do not have an upvote ratio
        # see https://api.reddit.com/api/info/?id=t1_eg7dfxx

# iterate over ids and check which ids are not in updated => for those we set the scores to None
# we check updated but modify updated_new
print("Original scores lst size {0}, ids size {1} => need to add {2} entries".format(len(scores_lst), len(ids), abs(len(scores_lst)-len(ids))))

100%|█████████▉| 618913/618918 [3:34:03<00:00, 48.19it/s]

Original scores lst size 618913, ids size 618918 => need to add 5 entries


In [14]:
import helpers.helper_functions as helper_functions

In [51]:
    insertion_idxs = []
    if len(scores_lst) != len(ids):
        for i in range(len(ids)):
            if i < len(scores_lst):
                id_orig = ids[i]
                id_updated = scores_lst[i][0]
                if id_orig != id_updated:
                    insertion_idxs.append(i)
                    #empty_insert = [None] * len(scores_lst[i])
                    #empty_insert[0] = ids[i]
                    #scores_lst_new.insert(i, empty_insert)
            else:
                insertion_idxs.append(i)
        
        scores_lst_new = scores_lst.copy()
        
        for i in insertion_idxs:
            empty_insert = [None] * len(scores_lst[0])
            empty_insert[0] = ids[i]
            scores_lst_new.insert(i+insertion_idxs.index(i), empty_insert)

In [52]:
print(insertion_idxs)

[618913, 618914, 618915, 618916, 618917]


In [2]:
def get_ups_downs_from_ratio_score(r,s):
    """Given a ratio and a score, we will return individual number of upvotes and downvotes & ratio, score
        We have to do this since, reddit remove the possibility to check the exact amount of upvotes and downvotes a few years ago. 
        Also the ratio is rounded so we will not get completely accurate values.

        We get formula since we know that x-y=s and x/x+y = r

    Args:
        r (float): upvote ratio from reddit
        s (int): the current score of the post/comment

    Returns:
        list: list with first element being the number of upvotes and the second one the number of downvotes, followed by the upvote ratio and current score
    """
    if r is None or s is None:
        print("NONE")
        return [None, None, None, None]
    
    ups = round((r * s) / ( 2 * r - 1 )) if r != 0.5 else 0 #if we have have a 50% upvote ratio we set the upvotes to 0
    downs = round(ups - s)
    return [ups, downs, r, s]

In [7]:
print(get_ups_downs_from_ratio_score(0.62,2))

[5, 3, 0.62, 2]


In [56]:
scores_np = np.array(scores_lst_new)
scores = scores_np[:,1]
if do_posts:
    ratios = scores_np[:,2]
    ups_downs = list(map(lambda s, r: get_ups_downs_from_ratio_score(r, s) , scores, ratios))
    ups_downs = np.array(ups_downs)
    df[prefix+"_ups"] = ups_downs[:,0].tolist()
    df[prefix+"_downs"] = ups_downs[:,1].tolist()
    df[prefix+"_ratio"] = ups_downs[:,2].tolist()
    df[prefix+"_score"] = ups_downs[:,3].tolist()
    df[prefix+"_flair"] = scores_np[:,3].tolist()
else:
    df[prefix+"_score"] = scores
    

save_dir = CS.POSTS_CLEAN if do_posts else CS.COMMENTS_CLEAN
if not overwrite:
    save_dir = save_dir.replace(".", "_updated.")
lg.info(f"Saving updated {prefix}s to {save_dir}")
df.to_csv(save_dir, index=False)

NONE
NONE
NONE
NONE
NONE


2021-11-13 10:31:37 PHDesktop root[1632] INFO Saving updated posts to /home/philipp/Document/Coding/Reddit_Morality_Analysis/datasets/data/posts_cleaned_27_6_2021.csv


(618918, 11)


In [57]:
print(df.shape)
print(len(ids))

(618918, 11)
618918


In [58]:
df.isnull().sum(axis = 0)

post_id                   0
post_text                 0
post_title                0
post_author_id            0
post_score                5
post_created_utc          0
post_num_comments         0
post_ups                  5
post_downs                5
post_ratio                5
post_flair           265549
dtype: int64